In [ ]:
# stdlib
import logging
from pathlib import Path
from pprint import pprint
import random
import sys
import time

# third party
import veilid

project_root = Path.cwd().parent.parent.parent
veilid_path = project_root / "packages" / "grid" / "veilid"
sys.path.append(veilid_path.as_posix())

# third party
from server.veilid_streamer import VeilidStreamer  # type: ignore

logging.getLogger('server.veilid_streamer').setLevel(logging.DEBUG)

In [ ]:
host = "localhost"
port = 5960

In [ ]:
async def noop_callback(update: veilid.VeilidUpdate):
    pass


conn = await veilid.json_api_connect(host, port, noop_callback)
time.sleep(2)  # hack: wait for the connection to be established

In [ ]:
router = await (await conn.new_routing_context()).with_default_safety()
dht_key_str = input("Enter DHT Key of the receiver: ")
dht_key = veilid.TypedKey(dht_key_str.lstrip("VLD0:"))
try:
    await router.close_dht_record(dht_key)
except Exception:
    pass
await router.open_dht_record(key=dht_key, writer=None)
record_value = await router.get_dht_value(key=dht_key, subkey=0, force_refresh=True)
private_route = await conn.import_remote_private_route(record_value.data)
await router.app_call(private_route, b"Ready!")

In [ ]:
vs = VeilidStreamer(connection=conn, router=router)

async def send_random_message(message_size_kb):
    message = random.randbytes(message_size_kb * 1024)
    print(f"Sending message of size {len(message) // 1024} KB...")
    start = time.time()
    response = await vs.stream(private_route, message)
    end = time.time()
    print(f"[{end - start}s] Response: {response}")
    time_taken = end - start
    return time_taken


In [ ]:
# Tests with smaller messages
benchmarks = {}
for message_size_kb in range(0, 13):  # Test from 1 KB to 4 MB
    message_size_kb = 2**message_size_kb
    benchmarks[message_size_kb] = await send_random_message(message_size_kb)

pprint(benchmarks)

In [ ]:
await router.app_call(private_route, b"QUIT")